In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')

This notebook is a guide for the neural networks based models of the library of the project "". :)

Météo-France, CNRS, CNRM, Centre d’Etudes de la Neige \
Léo Catteau, Spring 2025

### Contents:
* [Training](#first-bullet)
* [Imputation](#second-bullet)

All neural networks based models are trained following the same procedure implemented by the Filler. 

### Training <a class="anchor" id="first-bullet"></a>

TBC